In [12]:
import torch, pickle
import numpy as np
from collections import Counter
import os, tqdm
from collections import defaultdict
from pathlib import Path

We want to have (as much as possible) balanced input combinations

Step 1. figure out the frequency of each combination 

Step 2. Decide the rate to drop

In [13]:
def filter_1d(x):
    
    if os.path.exists(x) == False:
        return False
    c_tensor, h_tensor = torch.load(x)
    return len(c_tensor)>0 or len(h_tensor)>0

def filter_H(x):
    if os.path.exists(x) == False:
        return False
    c_tensor, h_tensor = torch.load(x)
    return len(h_tensor)>0

def filter_C(x):
    if os.path.exists(x) == False:
        return False
    c_tensor, h_tensor = torch.load(x)
    return len(c_tensor)>0 

def filter_both(x):
    if os.path.exists(x) == False:
        return False
    c_tensor, h_tensor = torch.load(x)
    return len(c_tensor)>0 and len(h_tensor)>0

In [14]:
'''Total molecules'''
for split in ['train', 'val', 'test']:
    print(f'\n{split}')
    path_2d = Path(f'/workspace/SMILES_dataset/{split}')
    path_1d = Path(f'/workspace/OneD_Only_Dataset/{split}')
    hsqc_2d = os.listdir(path_2d/'HSQC')
    oneD_NMRs = os.listdir(path_1d/'oneD_NMR')
    
    oneds_that_merged_with_2d = os.listdir(path_2d/'oneD_NMR')
    only_c = list(filter(filter_C, [ path_2d/'oneD_NMR'/f  for f in hsqc_2d])) + list(filter(filter_C,[ path_1d/'oneD_NMR'/f  for f in oneD_NMRs]))
    only_h = list(filter(filter_H, [ path_2d/'oneD_NMR'/f  for f in hsqc_2d])) + list(filter(filter_H,[ path_1d/'oneD_NMR'/f  for f in oneD_NMRs]))
    both_1d = list(filter(filter_both, [ path_2d/'oneD_NMR'/f  for f in hsqc_2d])) + list(filter(filter_both,[ path_1d/'oneD_NMR'/f  for f in oneD_NMRs]))
    print(f'Total 2D molecules: {len(hsqc_2d)}')
    print(f'Total num molecules that have 1D NMR: {len(oneD_NMRs)} + {len(oneds_that_merged_with_2d)} = {len(oneD_NMRs) + len(oneds_that_merged_with_2d)}')
    print(f" --This means that {len(oneds_that_merged_with_2d)} molecules have 2D NMRs and {len(oneD_NMRs)} don't")
    print("Inside molecules that have 1D NMR:")
    print(f'    have C: {len(only_c)}')
    print(f'    have H: {len(only_h)}')
    print(f'    have both 1d: {len(both_1d)}')
    
    
    only_hsqc, hsqc_and_c, hsqc_and_h, all_3 = 0, 0, 0, 0
    empty = []
    for i in hsqc_2d:
        # if file exists
        if os.path.exists(path_2d / 'oneD_NMR'/ i):
            c_tensor, h_tensor = torch.load(path_2d / 'oneD_NMR'/ i)
            if len(c_tensor) > 0 and len(h_tensor) > 0:
                all_3 += 1
            elif len(c_tensor) > 0:
                hsqc_and_c += 1
            elif len(h_tensor) > 0:
                hsqc_and_h += 1
            else:
                empty.append(i)
        else:
            only_hsqc += 1
    # print(f'Only HSQC: {only_hsqc} | HSQC and C: {hsqc_and_c} | HSQC and H: {hsqc_and_h} | All 3: {all_3} | Empty: {len(empty)}')
    # print('which means we have the following available:')
    print("Inside molecules that have HSQC:")
    print(f'  HSQC and C: {hsqc_and_c+all_3} | HSQC and H: {hsqc_and_h+all_3} | All 3: {all_3} | Empty: {len(empty)}')
    
    print(f'Total molecules: {len(hsqc_2d) + len(oneD_NMRs)}')


train
Total 2D molecules: 109694
Total num molecules that have 1D NMR: 63728 + 39686 = 103414
 --This means that 39686 molecules have 2D NMRs and 63728 don't
Inside molecules that have 1D NMR:
    have C: 103409
    have H: 93374
    have both 1d: 93370
Inside molecules that have HSQC:
  HSQC and C: 39685 | HSQC and H: 33203 | All 3: 33203 | Empty: 1
Total molecules: 173422

val
Total 2D molecules: 13738
Total num molecules that have 1D NMR: 7883 + 4910 = 12793
 --This means that 4910 molecules have 2D NMRs and 7883 don't
Inside molecules that have 1D NMR:
    have C: 12792
    have H: 11482
    have both 1d: 11481
Inside molecules that have HSQC:
  HSQC and C: 4910 | HSQC and H: 4056 | All 3: 4056 | Empty: 0
Total molecules: 21621

test
Total 2D molecules: 13630
Total num molecules that have 1D NMR: 7914 + 4865 = 12779
 --This means that 4865 molecules have 2D NMRs and 7914 don't
Inside molecules that have 1D NMR:
    have C: 12779
    have H: 11500
    have both 1d: 11500
Inside mol

In [15]:
for split in [ 'val', 'test', 'train']:
    print(f'\n{split}')
    path_2d = Path(f'/workspace/SMILES_dataset/{split}')
    path_1d = Path(f'/workspace/OneD_Only_Dataset/{split}')
    hsqc_2d = os.listdir(path_2d/'HSQC')
    oneD_NMRs = os.listdir(path_1d/'oneD_NMR')
    
    only_hsqc, hsqc_and_c, hsqc_and_h, all_3 = 0, 0, 0, 0
    empty = []
    for i in hsqc_2d:
        # if file exists
        if os.path.exists(path_2d / 'oneD_NMR'/ i):
            c_tensor, h_tensor = torch.load(path_2d / 'oneD_NMR'/ i)
            if len(c_tensor) > 0 and len(h_tensor) > 0:
                all_3 += 1
            elif len(c_tensor) > 0:
                hsqc_and_c += 1
            elif len(h_tensor) > 0:
                hsqc_and_h += 1
            else:
                empty.append(i)
        else:
            only_hsqc += 1
    # print(f'Only HSQC: {only_hsqc} | HSQC and C: {hsqc_and_c} | HSQC and H: {hsqc_and_h} | All 3: {all_3} | Empty: {len(empty)}')
    # print('which means we have the following available:')
    print(f' HSQC and C: {hsqc_and_c+all_3} | HSQC and H: {hsqc_and_h+all_3} | All 3: {all_3} | Empty: {len(empty)}')
        


val
 HSQC and C: 4910 | HSQC and H: 4056 | All 3: 4056 | Empty: 0

test
 HSQC and C: 4865 | HSQC and H: 4056 | All 3: 4056 | Empty: 0

train
 HSQC and C: 39685 | HSQC and H: 33203 | All 3: 33203 | Empty: 1


In [16]:
"""only both 1d"""
only_c, only_h, c_and_h = 0, 0, 0
empty = []
for i in oneD_NMRs:
    c_tensor, h_tensor = torch.load(path_1d / 'oneD_NMR'/ i)
    if len(c_tensor) > 0 and len(h_tensor) > 0:
        c_and_h += 1
    elif len(c_tensor) > 0:
        only_c += 1
    elif len(h_tensor) > 0:
        only_h += 1
    else:
        empty.append(i)
        
print(f'Only C: {only_c} | Only H: {only_h} | Both C and H: {c_and_h}')
print(len(empty))
    

Only C: 3557 | Only H: 4 | Both C and H: 60167
0


In [17]:
summary = {
    'only_hsqc': only_hsqc,
    'hsqc_and_c': hsqc_and_c,
    'hsqc_and_h': hsqc_and_h,
    'all_3': all_3,
    'only_c': only_c,
    'only_h': only_h,
    'c_and_h': c_and_h
}
summary

{'only_hsqc': 70008,
 'hsqc_and_c': 6482,
 'hsqc_and_h': 0,
 'all_3': 33203,
 'only_c': 3557,
 'only_h': 4,
 'c_and_h': 60167}

In [18]:
#solve the equation:
# hsqc_and_c + hsqc_and_c_from_all =  hsqc_and_h + hsqc_and_h_from_all
# hsqc_and_h + hsqc_and_h_from_all =  all_3 - hsqc_and_c_from_all - hsqc_and_h_from_all


# 2 * (hsqc_and_h + hsqc_and_h_from_all) = all_3 + hsqc_and_c - hsqc_and_h_from_all
hsqc_and_h_from_all = int(( all_3 +  hsqc_and_c - 2 * hsqc_and_h) / 3)
hsqc_and_c_from_all = hsqc_and_h + hsqc_and_h_from_all - hsqc_and_c 

In [19]:
print( hsqc_and_c + hsqc_and_c_from_all, hsqc_and_h + hsqc_and_h_from_all, all_3 - hsqc_and_c_from_all - hsqc_and_h_from_all)
# assert(hsqc_and_c + hsqc_and_c_from_all == hsqc_and_h + hsqc_and_h_from_all == all_3 - hsqc_and_c_from_all - hsqc_and_h_from_all)
print(f"In 2D: drop H rate is {hsqc_and_c_from_all / (all_3)}")
print(f"In 2D: drop C rate is {hsqc_and_h_from_all / (all_3)}")

13228 13228 13229
In 2D: drop H rate is 0.20317441195072736
In 2D: drop C rate is 0.3983977351444147


In [20]:
H_from_oned = int((c_and_h + only_c - 2 * only_h) / 3)
C_from_oned = only_h + H_from_oned - only_c

In [21]:
print(only_c + C_from_oned, only_h + H_from_oned, c_and_h - C_from_oned - H_from_oned)
print(f"In 1D: drop H rate is {C_from_oned / (c_and_h)}")
print(f"In 1D: drop C rate is {H_from_oned / (c_and_h)}")

21242 21242 21244
In 1D: drop H rate is 0.293931889574019
In 1D: drop C rate is 0.3529841939933851


In [22]:
total_count = sum(summary.values()) 
print(f"all3, hsqc_and_c, hsqc_and_h rate:  is", (hsqc_and_c + hsqc_and_c_from_all)/total_count)
print(f"only_c, only_h, c_and_c rate is", (only_c + C_from_oned)/total_count)
print(f"only_hsqc rate is", only_hsqc/total_count)

all3, hsqc_and_c, hsqc_and_h rate:  is 0.07627680615381068
only_c, only_h, c_and_c rate is 0.12248804931352028
only_hsqc rate is 0.4036881346549726
